In [ ]:
import os

folder_path = './image'  

files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

files.sort()

for i, filename in enumerate(files, start=1):
    extension = os.path.splitext(filename)[1]
    
    new_name = f"{i}{extension}"
    
    old_path = os.path.join(folder_path, filename)
    new_path = os.path.join(folder_path, new_name)
    
    os.rename(old_path, new_path)

print("เปลี่ยนชื่อเสร็จแล้ว!")


FileExistsError: [WinError 183] Cannot create a file when that file already exists: './image\\10.JPG' -> './image\\2.JPG'

: 

In [3]:
import os
import cv2
import math
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk
import sys


class RectangleDrawer:
    def __init__(self, root, image_path):
        self.root = root
        self.image_path = image_path
        self.original_image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

        self.orig_height, self.orig_width = self.original_image.shape[:2]
        max_width, max_height = 1800, 900
        scale_w = max_width / self.orig_width
        scale_h = max_height / self.orig_height
        self.scale = min(scale_w, scale_h, 1.0)

        self.display_width = int(self.orig_width * self.scale)
        self.display_height = int(self.orig_height * self.scale)
        self.display_image = cv2.resize(self.original_image, (self.display_width, self.display_height)) if self.scale < 1.0 else self.original_image.copy()
        self.clone = self.original_image.copy()

        self.marker_color = '#1E90FF'  
        self.marker_outline = '#0047AB'  
        self.marker_width = 3
        
        self.axis_color = '#FF4500'  
        self.axis_outline = '#8B0000'  
        self.axis_width = 2
        
        self.shift_held = False
        self.ctrl_held = False
        
        self.drawing_mode = 'rectangle'  
        
        self.rect_id = None
        self.rect_coords = None
        
        self.x_axis_id = None
        self.y_axis_id = None
        self.x_axis_coords = None
        self.y_axis_coords = None
        
        self.start_x = self.start_y = None
        self.drag_data = {"x": 0, "y": 0}
        self.is_dragging = False
        self.is_drawing = False
        self.can_move = False

        self.setup_ui()
        self.bind_events()

    def setup_ui(self):
        self.main_frame = Frame(self.root, bg='#f0f0f0')
        self.main_frame.pack(side=TOP, fill=BOTH, expand=True, padx=10, pady=5)

        self.canvas_frame = Frame(self.main_frame)
        self.canvas_frame.pack(fill=BOTH, expand=True)

        self.canvas = Canvas(
            self.canvas_frame, 
            width=self.display_width, 
            height=self.display_height,
            bg='white',
            relief=SUNKEN,
            bd=2,
            cursor="crosshair"
        )
        self.canvas.pack()

        self.pil_image = Image.fromarray(self.display_image)
        self.tk_image = ImageTk.PhotoImage(self.pil_image)
        self.canvas_image = self.canvas.create_image(0, 0, anchor=NW, image=self.tk_image)

        self.control_frame = Frame(self.root, bg='#e0e0e0', relief=RAISED, bd=1)
        self.control_frame.pack(side=TOP, fill=X, padx=10, pady=5)

        self.mode_frame = Frame(self.control_frame, bg='#e0e0e0')
        self.mode_frame.pack(side=LEFT, pady=5)

        mode_style = {
            'font': ('Arial', 9, 'bold'),
            'relief': RAISED,
            'bd': 2,
            'padx': 10,
            'pady': 3
        }

        self.mode_var = StringVar(value='rectangle')
        
        Radiobutton(self.mode_frame, text="🔲 Rectangle", variable=self.mode_var, 
                   value='rectangle', command=self.change_mode,
                   bg='#4CAF50', fg='white', selectcolor='#2E7D32', **mode_style).pack(side=LEFT, padx=2)
        
        Radiobutton(self.mode_frame, text="📏 X-Axis (3cm)", variable=self.mode_var, 
                   value='x_axis', command=self.change_mode,
                   bg='#FF9800', fg='white', selectcolor='#E65100', **mode_style).pack(side=LEFT, padx=2)
        
        Radiobutton(self.mode_frame, text="📐 Y-Axis (3cm)", variable=self.mode_var, 
                   value='y_axis', command=self.change_mode,
                   bg='#FF9800', fg='white', selectcolor='#E65100', **mode_style).pack(side=LEFT, padx=2)

        self.btn_frame = Frame(self.control_frame, bg='#e0e0e0')
        self.btn_frame.pack(side=LEFT, pady=5, padx=20)

        button_style = {
            'font': ('Arial', 10, 'bold'),
            'relief': RAISED,
            'bd': 2,
            'padx': 15,
            'pady': 5
        }

        Button(self.btn_frame, text="🎯 Auto Generate", command=self.auto_generate_rectangle, 
               bg='#9C27B0', fg='white', **button_style).pack(side=LEFT, padx=5)
        Button(self.btn_frame, text="💾 Save (Ctrl+S)", command=self.save_image, 
               bg='#4CAF50', fg='white', **button_style).pack(side=LEFT, padx=5)
        Button(self.btn_frame, text="🗑️ Clear", command=self.clear_all, 
               bg='#f44336', fg='white', **button_style).pack(side=LEFT, padx=5)
        Button(self.btn_frame, text="⏭️ Skip", command=self.skip_image, 
               bg='#2196F3', fg='white', **button_style).pack(side=LEFT, padx=5)
        Button(self.btn_frame, text="❌ Quit", command=self.quit_program, 
               bg='#9e9e9e', fg='white', **button_style).pack(side=LEFT, padx=5)

        self.info_frame = Frame(self.control_frame, bg='#e0e0e0')
        self.info_frame.pack(side=RIGHT, pady=5)

        self.status_label = Label(
            self.info_frame, 
            text="Select mode: Rectangle for manual drawing | X/Y-Axis for 3cm reference lines | Auto Generate to create square",
            font=('Arial', 10),
            bg='#e0e0e0',
            fg='#333'
        )
        self.status_label.pack(side=RIGHT, padx=10)

    def bind_events(self):
        self.root.bind("<KeyPress-Shift_L>", lambda e: self.set_shift(True))
        self.root.bind("<KeyRelease-Shift_L>", lambda e: self.set_shift(False))
        self.root.bind("<KeyPress-Shift_R>", lambda e: self.set_shift(True))
        self.root.bind("<KeyRelease-Shift_R>", lambda e: self.set_shift(False))
        self.root.bind("<KeyPress-Control_L>", lambda e: self.set_ctrl(True))
        self.root.bind("<KeyRelease-Control_L>", lambda e: self.set_ctrl(False))
        self.root.bind("<KeyPress-Control_R>", lambda e: self.set_ctrl(True))
        self.root.bind("<KeyRelease-Control_R>", lambda e: self.set_ctrl(False))
        
        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<B1-Motion>", self.on_move_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)
        self.canvas.bind("<Motion>", self.on_mouse_move)
        
        self.root.bind("<Control-s>", lambda e: self.save_image())
        self.root.bind("<Delete>", lambda e: self.clear_all())
        self.root.bind("<BackSpace>", lambda e: self.clear_all())
        self.root.bind("<Control-Right>", lambda e: self.skip_image())
        self.root.bind("<Control-q>", lambda e: self.quit_program())
        self.root.bind("<Control-g>", lambda e: self.auto_generate_rectangle())
        
        self.root.bind("1", lambda e: self.set_mode('rectangle'))
        self.root.bind("2", lambda e: self.set_mode('x_axis'))
        self.root.bind("3", lambda e: self.set_mode('y_axis'))

        self.root.focus_set()

    def set_shift(self, state):
        self.shift_held = state

    def set_ctrl(self, state):
        self.ctrl_held = state

    def change_mode(self):
        self.drawing_mode = self.mode_var.get()
        self.update_cursor()
        self.update_status_for_mode()

    def set_mode(self, mode):
        self.mode_var.set(mode)
        self.change_mode()

    def update_cursor(self):
        if self.drawing_mode == 'rectangle':
            self.canvas.config(cursor="crosshair")
        elif self.drawing_mode in ['x_axis', 'y_axis']:
            self.canvas.config(cursor="pencil")

    def update_status_for_mode(self):
        if self.drawing_mode == 'rectangle':
            self.status_label.config(text="Rectangle mode: Click and drag to draw | Hold Shift for square | Click inside to move")
        elif self.drawing_mode == 'x_axis':
            self.status_label.config(text="X-Axis mode: Draw horizontal line representing 3cm width")
        elif self.drawing_mode == 'y_axis':
            self.status_label.config(text="Y-Axis mode: Draw vertical line representing 3cm height")

    def update_status(self, message):
        self.status_label.config(text=message)

    def on_mouse_move(self, event):
        if self.drawing_mode == 'rectangle':
            if self.rect_id and self.is_inside_rect(event.x, event.y):
                self.canvas.config(cursor="hand2")
                self.can_move = True
            else:
                self.canvas.config(cursor="crosshair")
                self.can_move = False

    def on_button_press(self, event):
        if self.drawing_mode == 'rectangle':
            self.handle_rectangle_press(event)
        elif self.drawing_mode == 'x_axis':
            self.handle_x_axis_press(event)
        elif self.drawing_mode == 'y_axis':
            self.handle_y_axis_press(event)

    def handle_rectangle_press(self, event):
        if self.rect_id and self.is_inside_rect(event.x, event.y):
            self.is_dragging = True
            self.drag_data["x"] = event.x
            self.drag_data["y"] = event.y
            self.canvas.config(cursor="hand2")
            self.update_status("Moving rectangle... Release to place")
        else:
            if self.rect_id:
                self.canvas.delete(self.rect_id)
                self.rect_id = None
                self.rect_coords = None
            
            self.start_x = event.x
            self.start_y = event.y
            self.is_drawing = True
            
            self.rect_id = self.canvas.create_rectangle(
                self.start_x, self.start_y, self.start_x, self.start_y,
                fill=self.marker_color,
                outline=self.marker_outline,
                width=self.marker_width,
                stipple='gray25' 
            )
            self.update_status("Drawing rectangle...")

    def handle_x_axis_press(self, event):
        if self.x_axis_id:
            self.canvas.delete(self.x_axis_id)
            self.x_axis_id = None
            self.x_axis_coords = None
        
        self.start_x = event.x
        self.start_y = event.y
        self.is_drawing = True
        
        self.x_axis_id = self.canvas.create_line(
            self.start_x, self.start_y, self.start_x, self.start_y,
            fill=self.axis_color,
            width=self.axis_width
        )
        self.update_status("Drawing X-axis (3cm reference)...")

    def handle_y_axis_press(self, event):
        if self.y_axis_id:
            self.canvas.delete(self.y_axis_id)
            self.y_axis_id = None
            self.y_axis_coords = None
        
        self.start_x = event.x
        self.start_y = event.y
        self.is_drawing = True
        
        self.y_axis_id = self.canvas.create_line(
            self.start_x, self.start_y, self.start_x, self.start_y,
            fill=self.axis_color,
            width=self.axis_width
        )
        self.update_status("Drawing Y-axis (3cm reference)...")

    def on_move_press(self, event):
        if self.drawing_mode == 'rectangle':
            self.handle_rectangle_move(event)
        elif self.drawing_mode == 'x_axis':
            self.handle_x_axis_move(event)
        elif self.drawing_mode == 'y_axis':
            self.handle_y_axis_move(event)

    def handle_rectangle_move(self, event):
        if self.is_drawing and self.rect_id:
            x1, y1 = self.start_x, self.start_y
            x2, y2 = event.x, event.y

            if self.shift_held:
                dx = x2 - x1
                dy = y2 - y1
                side = min(abs(dx), abs(dy))
                x2 = x1 + side if dx >= 0 else x1 - side
                y2 = y1 + side if dy >= 0 else y1 - side

            self.canvas.coords(self.rect_id, x1, y1, x2, y2)
            
        elif self.is_dragging and self.rect_id:
            dx = event.x - self.drag_data["x"]
            dy = event.y - self.drag_data["y"]
            self.canvas.move(self.rect_id, dx, dy)
            self.drag_data["x"] = event.x
            self.drag_data["y"] = event.y

    def handle_x_axis_move(self, event):
        if self.is_drawing and self.x_axis_id:
            self.canvas.coords(self.x_axis_id, self.start_x, self.start_y, event.x, event.y)

    def handle_y_axis_move(self, event):
        if self.is_drawing and self.y_axis_id:
            self.canvas.coords(self.y_axis_id, self.start_x, self.start_y, event.x, event.y)

    def on_button_release(self, event):
        if self.drawing_mode == 'rectangle':
            self.handle_rectangle_release(event)
        elif self.drawing_mode == 'x_axis':
            self.handle_x_axis_release(event)
        elif self.drawing_mode == 'y_axis':
            self.handle_y_axis_release(event)

    def handle_rectangle_release(self, event):
        if self.is_drawing and self.rect_id:
            coords = self.canvas.coords(self.rect_id)
            if abs(coords[2] - coords[0]) > 10 and abs(coords[3] - coords[1]) > 10:  # Minimum size
                self.rect_coords = coords
                self.update_status("Rectangle created! Click inside to move it, or click outside to create new one")
            else:
                self.canvas.delete(self.rect_id)
                self.rect_id = None
                self.rect_coords = None
                self.update_status("Rectangle too small - try again")
            
            self.is_drawing = False
            
        elif self.is_dragging:
            if self.rect_id:
                self.rect_coords = self.canvas.coords(self.rect_id)
                self.update_status("Rectangle moved! Click inside to move again, or click outside to create new one")
            self.is_dragging = False

    def handle_x_axis_release(self, event):
        if self.is_drawing and self.x_axis_id:
            coords = self.canvas.coords(self.x_axis_id)
            if abs(coords[2] - coords[0]) > 20:  
                self.x_axis_coords = coords
                self.update_status("X-axis created! Now draw Y-axis or use Auto Generate")
            else:
                self.canvas.delete(self.x_axis_id)
                self.x_axis_id = None
                self.x_axis_coords = None
                self.update_status("X-axis too short - try again")
            self.is_drawing = False

    def handle_y_axis_release(self, event):
        if self.is_drawing and self.y_axis_id:
            coords = self.canvas.coords(self.y_axis_id)
            if abs(coords[3] - coords[1]) > 20:  
                self.y_axis_coords = coords
                self.update_status("Y-axis created! Now draw X-axis or use Auto Generate")
            else:
                self.canvas.delete(self.y_axis_id)
                self.y_axis_id = None
                self.y_axis_coords = None
                self.update_status("Y-axis too short - try again")
            self.is_drawing = False

    def auto_generate_rectangle(self):
        if not self.x_axis_coords or not self.y_axis_coords:
            messagebox.showwarning("Missing Reference", "Please draw both X-axis and Y-axis reference lines first!")
            return

        x1, y1, x2, y2 = self.x_axis_coords
        x_length = math.hypot(x2 - x1, y2 - y1)  

        x1, y1, x2, y2 = self.y_axis_coords
        y_length = math.hypot(x2 - x1, y2 - y1)

        x_pixels_per_cm = x_length / 3.0
        y_pixels_per_cm = y_length / 3.0

        pixels_per_cm = min(x_pixels_per_cm, y_pixels_per_cm)
        square_size = pixels_per_cm * 3 

        ref_x, ref_y = self.line_intersection(self.x_axis_coords, self.y_axis_coords)
        if ref_x is None or ref_y is None:
            x_axis_center_x = (self.x_axis_coords[0] + self.x_axis_coords[2]) / 2
            x_axis_center_y = (self.x_axis_coords[1] + self.x_axis_coords[3]) / 2
            y_axis_center_x = (self.y_axis_coords[0] + self.y_axis_coords[2]) / 2
            y_axis_center_y = (self.y_axis_coords[1] + self.y_axis_coords[3]) / 2
            ref_x = (x_axis_center_x + y_axis_center_x) / 2
            ref_y = (x_axis_center_y + y_axis_center_y) / 2

        half_size = square_size / 2
        square_x1 = ref_x - half_size
        square_y1 = ref_y - half_size
        square_x2 = ref_x + half_size
        square_y2 = ref_y + half_size

        if self.rect_id:
            self.canvas.delete(self.rect_id)

        self.rect_id = self.canvas.create_rectangle(
            square_x1, square_y1, square_x2, square_y2,
            fill=self.marker_color,
            outline=self.marker_outline,
            width=self.marker_width,
            stipple='gray25'
        )
        self.rect_coords = [square_x1, square_y1, square_x2, square_y2]
        self.set_mode('rectangle')
        self.update_status(f"Auto-generated 3x3cm square! (X: {x_pixels_per_cm:.1f} px/cm, Y: {y_pixels_per_cm:.1f} px/cm)")
        
    def line_intersection(self, line1, line2):
        x1, y1, x2, y2 = line1
        x3, y3, x4, y4 = line2

        denom = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
        if denom == 0:
            return None, None  

        px = ((x1*y2 - y1*x2)*(x3 - x4) - (x1 - x2)*(x3*y4 - y3*x4)) / denom
        py = ((x1*y2 - y1*x2)*(y3 - y4) - (y1 - y2)*(x3*y4 - y3*x4)) / denom
        return px, py
        
    def is_inside_rect(self, x, y):
        if not self.rect_coords:
            return False
        x1, y1, x2, y2 = self.rect_coords

        min_x, max_x = min(x1, x2), max(x1, x2)
        min_y, max_y = min(y1, y2), max(y1, y2)
        return min_x <= x <= max_x and min_y <= y <= max_y

    def clear_all(self):
        """Clear all elements"""
        cleared_items = []
        
        if self.rect_id:
            self.canvas.delete(self.rect_id)
            self.rect_id = None
            self.rect_coords = None
            cleared_items.append("rectangle")
        
        if self.x_axis_id:
            self.canvas.delete(self.x_axis_id)
            self.x_axis_id = None
            self.x_axis_coords = None
            cleared_items.append("X-axis")
        
        if self.y_axis_id:
            self.canvas.delete(self.y_axis_id)
            self.y_axis_id = None
            self.y_axis_coords = None
            cleared_items.append("Y-axis")
        
        if cleared_items:
            self.update_status(f"Cleared: {', '.join(cleared_items)}")
        else:
            self.update_status("Nothing to clear")

    def save_image(self):
        if not self.rect_id or not self.rect_coords:
            messagebox.showwarning("No Rectangle", "Please draw a rectangle first!")
            return

        x1, y1, x2, y2 = map(int, self.rect_coords)

        min_x, max_x = min(x1, x2), max(x1, x2)
        min_y, max_y = min(y1, y2), max(y1, y2)

        real_x1 = int(min_x / self.scale)
        real_y1 = int(min_y / self.scale)
        real_x2 = int(max_x / self.scale)
        real_y2 = int(max_y / self.scale)

        real_x1 = max(0, min(real_x1, self.orig_width))
        real_y1 = max(0, min(real_y1, self.orig_height))
        real_x2 = max(0, min(real_x2, self.orig_width))
        real_y2 = max(0, min(real_y2, self.orig_height))

        cv2.rectangle(self.clone, (real_x1, real_y1), (real_x2, real_y2), 
                     (0, 0, 255), thickness=-1)  

        os.makedirs("image_out", exist_ok=True)
        filename = os.path.basename(self.image_path)
        save_path = os.path.join("image_out", filename)
        cv2.imwrite(save_path, cv2.cvtColor(self.clone, cv2.COLOR_RGB2BGR))
        
        messagebox.showinfo("Saved", f"Blue rectangle saved successfully!\n{save_path}")
        print(f"✅ Saved: {save_path}")
        self.root.destroy()

    def skip_image(self):
        """Skip current image without saving"""
        if messagebox.askyesno("Skip Image", "Skip this image without saving?"):
            print(f"⏭️ Skipped: {self.image_path}")
            self.root.destroy()

    def quit_program(self):
        if messagebox.askyesno("Quit", "Are you sure you want to quit?"):
            print("👋 Exiting...")
            self.root.destroy()
            sys.exit()

def process_all_images(folder_path):
    """Process all images in the folder from 1.jpg to 100.jpg"""
    for i in range(1, 101):
        image_file = os.path.join(folder_path, f"{i}.jpg")
        if os.path.exists(image_file):
            print(f"\n📂 Opening: {image_file}")

            root = Tk()
            root.title(f"Enhanced Rectangle Marker - Editing {i}.jpg ({i}/100)")
            root.geometry("1400x900")
            root.configure(bg='#f0f0f0')

            root.update_idletasks()
            x = (root.winfo_screenwidth() // 2) - (root.winfo_width() // 2)
            y = (root.winfo_screenheight() // 2) - (root.winfo_height() // 2)
            root.geometry(f"+{x}+{y}")
            
            app = RectangleDrawer(root, image_file)

            root.mainloop()
        else:
            print(f"❌ Not found: {image_file}")

if __name__ == "__main__":
    image_folder = "image"
    if not os.path.exists(image_folder):
        print(f"❌ Folder '{image_folder}' not found!")
        print("Please create the 'image' folder and add your images (1.jpg, 2.jpg, etc.)")
        sys.exit(1)
    
    print("🎨 Enhanced Rectangle Marker Tool - Auto-Generate Feature")
    print("=" * 70)
    print("NEW FEATURES:")
    print("• Auto-Generate: Create perfect 3x3cm square from axis references")
    print("• X-Axis Mode: Draw horizontal line representing 3cm")
    print("• Y-Axis Mode: Draw vertical line representing 3cm")
    print("=" * 70)
    print("WORKFLOW:")
    print("1. Switch to X-Axis mode (Orange button) and draw horizontal 3cm reference")
    print("2. Switch to Y-Axis mode (Orange button) and draw vertical 3cm reference")
    print("3. Click 'Auto Generate' to create perfect 3x3cm blue square")
    print("4. Use Rectangle mode to manually adjust if needed")
    print("5. Save the image")
    print("=" * 70)
    print("CONTROLS:")
    print("• Mode Selection: Radio buttons or keys 1,2,3")
    print("• Rectangle mode: Click and drag to draw | Hold Shift for square")
    print("• X/Y-Axis modes: Draw reference lines for 3cm measurements")
    print("• Auto Generate: Ctrl+G or purple button")
    print("• Save: Ctrl+S | Clear: Delete/Backspace | Skip: Ctrl+→ | Quit: Ctrl+Q")
    print("=" * 70)
    
    process_all_images(image_folder)
    

🎨 Enhanced Rectangle Marker Tool - Auto-Generate Feature
NEW FEATURES:
• Auto-Generate: Create perfect 3x3cm square from axis references
• X-Axis Mode: Draw horizontal line representing 3cm
• Y-Axis Mode: Draw vertical line representing 3cm
WORKFLOW:
1. Switch to X-Axis mode (Orange button) and draw horizontal 3cm reference
2. Switch to Y-Axis mode (Orange button) and draw vertical 3cm reference
3. Click 'Auto Generate' to create perfect 3x3cm blue square
4. Use Rectangle mode to manually adjust if needed
5. Save the image
CONTROLS:
• Mode Selection: Radio buttons or keys 1,2,3
• Rectangle mode: Click and drag to draw | Hold Shift for square
• X/Y-Axis modes: Draw reference lines for 3cm measurements
• Auto Generate: Ctrl+G or purple button
• Save: Ctrl+S | Clear: Delete/Backspace | Skip: Ctrl+→ | Quit: Ctrl+Q

📂 Opening: image\1.jpg
👋 Exiting...


SystemExit: 

C:\Users\ahmad\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
